In [9]:
'''
Delta-beta calculation script

Purpose: this script takes delta-beta output and calculates the 'combined impact' by adding together the three terms from this equation:

fulladapt_impact = HRshareFA*beta^H*(T_2099 - T_2015) + 
                 (1 - HRshareFA)*beta^L*(T_2099 - T_2015) + 
                 (HRshareFA- HRshareIA)*(beta^H - beta^L)*T_2015
                 
incadapt_impact = HRshareIA*beta^H*(T_2099 - T_2015) + 
                 (1 - HRshareIA)*beta^L*(T_2099 - T_2015)
                 
noadapt_impact = HRshareNA*beta^H*(T_2099 - T_2015) + 
                 (1 - HRshareNA)*beta^L*(T_2099 - T_2015) + 
                 (HRshareNA- HRshareIA)*(beta^H - beta^L)*T_2015

Parameters:

@model     :  the name of the model (and thus the name of folder that output is stored in)
@regions   :  a list of regions for which delta-beta CSVs have been outputted. They are generated from
                repos/gcp-labor/3_projection/deltabetas/yellow_purple_script_labor.R

'''
__author__ = 'Kit Schwarz'
__contact__ = 'csschwarz@uchicago.edu'
__version__ = '1.0'

############
# LIBRARIES
############

import xarray as xr
import pandas as pd
import getpass

############
# PARAMETERS
############

model = 'uninteracted_main_model'
# select: uninteracted_main_model_w_chn, uninteracted_main_model

regions = ['SDN.4.11.49.163', 'THA.3.R3edeff05b7928bfc', 'USA.5.221', 'CAN.3.50.1276', 'CAN.2.33.913', 'GBR.1.R7074136591e79d11'] 

region_dict = {
    'SDN.4.11.49.163' : 'Khartoum, SDN',
    'THA.3.R3edeff05b7928bfc' : 'Bangkok, THA', 
    'USA.5.221' : 'San Francisco, USA',
    'CAN.3.50.1276' : 'Winnipeg, CAN', 
    'CAN.2.33.913' : 'Vancouver, CAN', 
    'GBR.1.R7074136591e79d11' : 'London, GBR'
}

In [10]:


############
# PATHWAYS
############

username = getpass.getuser()
db_root = '/mnt/CIL_labor/3_projection/deltabetas/spline_27_37_39'

if model == 'uninteracted_main_model_w_chn':
    
    proj_root = '/shares/gcp/outputs/labor/impacts-woodwork/uninteracted_main_model_w_chn/uninteracted_splines_w_chn_21_37_41_by_risk_empshare_noFE_YearlyAverageDay/rcp85/CCSM4/high/SSP3'
    
elif model == 'uninteracted_main_model':
    
#     proj_root = '/shares/gcp/outputs/labor/impacts-woodwork/test_rcc/uninteracted_splines_27_37_39_by_risk_empshare_noFE_YearlyAverageDay/rcp85/CCSM4/high/SSP3'

    proj_root = '/shares/gcp/outputs/labor/impacts-woodwork/point_estimate_google_rebased/median/rcp85/CCSM4/high/SSP3'
    
else:
    
    print("Your model is unrecognized.")

In [11]:
############
# GET BETAS
############

high = [pd.read_csv(f'{db_root}/{model}/deltabeta_high{region}-2099.csv') for region in regions]
low = [pd.read_csv(f'{db_root}/{model}/deltabeta_low{region}-2099.csv')  for region in regions]

temp_bins = [df[['bin', 'T[2015]', 'T[diff]']] for df in high]

# subset columns and drop useless rows
subset_cols = ['bin', 'beta^F', 'beta^F*T[2099]-beta^I*T[2015]']
high = [df.loc[~df.bin.isin(['Total <20C', 'Total >20C', 'Total'])][subset_cols] for df in high]
low = [df.loc[~df.bin.isin(['Total <20C', 'Total >20C', 'Total'])][subset_cols] for df in low]

###################
# FUNCTION: get_lrt
###################

def get_climtas(regions, model, year=2099):
    
    df = xr.open_dataset(f"/shares/gcp/outputs/temps/rcp85/{model}/climtas.nc4").to_dataframe().reset_index()
    climtas = [df.loc[(df.year == year) & (df.regions == region)]['averaged'].iloc[0] for region in regions]
    
    return climtas

########################
# FUNCTION: get_loggdppc
########################

def get_loggdppc(regions, year=2099):
    
    df = pd.read_csv("/shares/gcp/outputs/labor/impacts-woodwork/test_rcc_copy/" +
        "uninteracted_splines_27_37_39_by_risk_empshare_noFE_YearlyAverageDay/rcp85/CCSM4/high/SSP3/" +
        "test_rcc_main_model_single_config-allcalcs-uninteracted_main_model.csv",
        skiprows=26, usecols=['region', 'year', 'climtas', 'loggdppc'])
    
    loggdppc = [df.loc[(df.year == year) & (df.region == region)]['loggdppc'].iloc[0] for region in regions]
    
    return loggdppc

############
# FUNCTION: get_shares
############
    
def get_shares(regions, adapt_list = ['fulladapt', 'incadapt', 'noadapt'], proj_root=proj_root, model=model):
    
    dict = {}
    
    for adapt in adapt_list:
        
        if adapt == "fulladapt":
            df = xr.open_dataset(f'{proj_root}/{model}.nc4').to_dataframe().reset_index()
        else:
            df = xr.open_dataset(f'{proj_root}/{model}-{adapt}.nc4').to_dataframe().reset_index()

        dict[f'{adapt}'] = [df.loc[(df.year == 2099) & (df.regions == region)]['clip'].iloc[0] for region in regions]
    
    return dict

############################
# FUNCTION: calc_delta_beta
############################

def calc_delta_beta(temp_bins, high, low, f_share, i_share, n_share):
    '''
    Calculates the fulladapt, incadapt, and noadapt impact for a given region.
    Input: a set of pd.DataFrames() containing the high and low betas, the temperature bins and days,
            and the high-risk share under fulladapt and incadapt scenarios.
            
    Output: a single pd.DataFrame().
    '''
    
    # rename columns according to risk group
    high.columns = [col + '_high' for col in high.columns]
    low.columns = [col + '_low' for col in low.columns]
    
    # merge together
    df = (temp_bins
                   .merge(high,
                          left_on='bin', right_on='bin_high')
                   .merge(low,
                          left_on='bin', right_on='bin_low')
              )
    
    # add in the incadapt and fulladapt shares
    df['fulladapt_share'] = f_share
    df['incadapt_share'] = i_share
    df['noadapt_share'] = n_share
    
    # calculating terms -> we do this for all the ref temps selected,
    # plus the basic delta beta with reference temperature 27C
    
    for adapt in 'fulladapt', 'incadapt', 'noadapt' :
        
        df[f'high_risk_term_{adapt}'] = df[f'{adapt}_share']      * df[f'beta^F_high'] * df['T[diff]']
        df[f'low_risk_term_{adapt}']  = (1- df[f'{adapt}_share']) * df[f'beta^F_low']  * df['T[diff]']
        df[f'extra_term_{adapt}']     = (df[f'{adapt}_share'] - df['incadapt_share']) * (df[f'beta^F_high'] - df[f'beta^F_low']) * df['T[2015]']
        df[f'comb_response_{adapt}']  = df[f'high_risk_term_{adapt}'] + df[f'low_risk_term_{adapt}'] + df[f'extra_term_{adapt}']
    
    return df

############################
# FUNCTION: get_daily_minutes
############################

def get_daily_minutes(df, cols):
    
    '''
    Calculates the daily average of a set of columns passed to cols.
    Input: a single pd.DataFrame(), and a list of cols.
            
    Output: a list of averages.
    '''
    
    numbers = [df[col].sum()/365 for col in cols]
    
    return numbers

In [12]:
############################
# GET RESULTS
############################

results = pd.DataFrame(index=regions)
results['name'] = [region_dict[key] for key in regions]

# get shares
shares = get_shares(regions=regions)

for key, value in zip(shares.keys(), shares.values()):
    results[f'{key}_Hshare'] = value
    

# get loggdppc
results['loggdppc'] = get_loggdppc(regions)

# get climtas - 2015
results['climtas_2015'] = get_climtas(regions, year=2015, model='CCSM4')

# get climtas - 2099
for climmodel in ['CCSM4', 'surrogate_GFDL-CM3_99']:
    results[f'climtas_2099_{climmodel}'] = get_climtas(regions, year=2099, model=climmodel)
    
# delta beta results
delta_beta = [calc_delta_beta(a,b,c,d,e,f) for a,b,c,d,e,f in zip(
    temp_bins, high, low, shares.get('fulladapt'), shares.get('incadapt'), shares.get('noadapt'))]

# convert certain columns to daily
daily_cols = ['comb_response_fulladapt', 'comb_response_incadapt', 'comb_response_noadapt', 'beta^F_high', 'beta^F_low', 'high_risk_term_fulladapt', 'low_risk_term_fulladapt', 'extra_term_fulladapt']
results_daily = [get_daily_minutes(df, daily_cols) for df in delta_beta]
daily_minutes = pd.DataFrame(results_daily, columns = daily_cols, index = regions)

results = results.merge(daily_minutes,
                       left_index=True,
                       right_index=True)



In [13]:
# export this baby
results.T

,SDN.4.11.49.163,THA.3.R3edeff05b7928bfc,USA.5.221,CAN.3.50.1276,CAN.2.33.913,GBR.1.R7074136591e79d11
name,"Khartoum, SDN","Bangkok, THA","San Francisco, USA","Winnipeg, CAN","Vancouver, CAN","London, GBR"
fulladapt_Hshare,0.710272,0.332272,0.0464441,0.10051,0.0673693,0.153267
incadapt_Hshare,0.511133,0.214898,0.0464441,0.0464441,0.130805,0.195679
noadapt_Hshare,0.769325,0.457875,0.142215,0.183243,0.283271,0.31001
loggdppc,9.52758,10.5961,11.4374,11.5021,11.5112,11.1678
climtas_2015,31.2349,29.332,15.071,3.87317,9.66988,10.2088
climtas_2099_CCSM4,34.9522,32.4208,17.9572,8.89141,14.0934,12.7255
climtas_2099_surrogate_GFDL-CM3_99,40.6749,37.2543,22.0846,15.4525,18.7994,18.6891
comb_response_fulladapt,-19.3994,-7.58546,0.188929,-0.416269,0.816314,0.596727
comb_response_incadapt,-12.102,-5.69568,0.188929,0.0624522,0.464538,0.296097
